In [ ]:
import warnings

import pandas as pd
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

warnings.filterwarnings("ignore")

In [ ]:
collection_id = "satellite-aerosol-properties"
request = {
    "format": "zip",
    "time_aggregation": "monthly_average",
    "variable": "aerosol_optical_depth",
    "sensor_on_satellite": "slstr_on_sentinel_3a",
    "algorithm": "ens",
    "year": [str(year) for year in range(2018, 2019 + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
    "version": "v2.1",
}

In [ ]:
def source_to_time(ds):
    time = pd.to_datetime(ds["source"].str.slice(None, 6), format="%Y%m")
    ds = ds.assign_coords(time=time)
    return ds.swap_dims(source="time")


def source_to_time_and_spatial_weighted_mean(ds):
    ds = source_to_time(ds)
    return diagnostics.spatial_weighted_mean(ds)


def source_to_time_and_spatial_weighted_std(ds):
    ds = source_to_time(ds)
    return diagnostics.spatial_weighted_std(ds)

In [ ]:
ds_mean = download.download_and_transform(
    collection_id,
    request,
    transform_func=source_to_time_and_spatial_weighted_mean,
    chunks={"year": 1},
)

In [ ]:
ds_std = download.download_and_transform(
    collection_id,
    request,
    transform_func=source_to_time_and_spatial_weighted_std,
    chunks={"year": 1},
)

In [ ]:
filename = "_".join(["global-timeseries", collection_id])
title = filename.replace("-", " ").replace("_", " ").title()
fig = plot.shaded_std("AOD550", ds_mean, ds_std, title=title)
fig.show()
fig.write_image(filename + ".png")